In [1]:
!pip install py7zr
!pip uninstall albumentations --yes
!pip install albumentations
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 1.9MB 6.7MB/s 
     |████████████████████████████████| 2.2MB 16.3MB/s 
     |████████████████████████████████| 122kB 43.0MB/s 
Uninstalling albumentations-0.1.12:
  Successfully uninstalled albumentations-0.1.12
     |████████████████████████████████| 81kB 3.9MB/s 
     |████████████████████████████████| 37.6MB 1.4MB/s 
     |████████████████████████████████| 952kB 42.6MB/s 
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-vn68ny_v
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-vn68ny_v
  Created wheel for albumentations: filename=albumentations-0.5.2-cp37-none-any.whl size=88144 sha256=21e10ee5bb4e17b973b52eccad4739679547cc9e74c3719c9316ec3d6607d345
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ho9gbyn/wheel

In [2]:
import requests
import py7zr
import sys
import os
from tqdm.notebook import tqdm
import zipfile
import yaml
from lxml import objectify
import pandas as pd
import cv2
from time import time
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GroupKFold, StratifiedKFold
import albumentations as A
import albumentations.pytorch.transforms
from albumentations.pytorch import ToTensor
#from albumentations.pytorch.transforms import ToTensorV2
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import nms
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
import datetime
# General
import warnings
warnings.filterwarnings("ignore")
import datetime
from time import time
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

# OpenCV
import cv2

# ScikitLearn for Data Splitting
from sklearn.model_selection import train_test_split, GroupKFold, StratifiedKFold

# Albumentations
import albumentations as A
#from albumentations.pytorch.transforms import ToTensorV2

# Pytorch
import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.ops import nms

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler


# Seed Everything for Reproducibility
SEED = 42

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(SEED)

In [3]:
def Download(name, url): #found this somewhere on stackoverflow
  """
  Function for downloading datasets (except downloading from Google Drive)
  """
  response = requests.get(url, stream=True)
  total_size_in_bytes= int(response.headers.get('content-length', 0))
  block_size = 1024 #1 Kibibyte
  progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
  with open('/content/zipdata/'+name, 'wb') as file:
      for data in response.iter_content(block_size):
          progress_bar.update(len(data))
          file.write(data)
  progress_bar.close()
  if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
      print("ERROR, something went wrong")

In [4]:
#Creating dirs to store datasets
try:
  os.makedirs('/content/zipdata')
  print('Directory "/content/zipdata" is created')
except:
  print('Directory "/content/zipdata" already exists')
try:
  os.makedirs('/content/unzipped')
  print('Directory "/content/unzipped" is created')
except:
  print('Directory "/content/unzipped" already exists')

Directory "/content/zipdata" is created
Directory "/content/unzipped" is created


In [5]:
#https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drive/39225039#39225039

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    total_size_in_bytes= int(response.headers.get('content-length', 0))
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

    save_response_content(response, destination, progress_bar)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination, progress_bar):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                progress_bar.update(len(chunk))


if __name__ == "__main__":
    file_id = '14aqK5W8mDkMY7OGD-iErzsKloRrTRw_x'
    destination = '/content/zipdata/bosch.7z'
    download_file_from_google_drive(file_id, destination)

6.04GiB [01:20, 94.5MiB/s]

In [6]:
with py7zr.SevenZipFile('/content/zipdata/bosch.7z', 'r') as archive:
    archive.extractall('/content/unzipped/BOSCH/')
with open('/content/unzipped/BOSCH/bosch/train.yaml') as f:
    templates = yaml.safe_load(f)
df_bosch = pd.json_normalize(templates, 'boxes', 'path')
try:
  df_bosch = df_bosch[df_bosch['occluded'] == False].drop(labels=['occluded'], axis=1)
except:
  None
df_bosch = df_bosch[df_bosch['label'].isin(['Yellow', 'Red', 'Green'])]
df_bosch.reset_index(drop=True, inplace=True)
df_bosch = df_bosch.rename(columns={'path':'Path', 'label':'Color', 'x_min':'x_min', 
                                  'y_min':'y_min', 'x_max':'x_max', 'y_max':'y_max'})
df_bosch = df_bosch.iloc[:,[-1,0,2,4,1,3]]
df_bosch = df_bosch.astype({'x_min':np.int64, 'y_min':np.int64, 'x_max':np.int64, 'y_max':np.int64})

for i in tqdm(range(df_bosch.shape[0])):
  df_bosch.loc[i,'Path'] = '/content/unzipped/BOSCH/bosch'+ df_bosch['Path'][i][1:]

6.05GiB [01:40, 94.5MiB/s]
100%|██████████| 8574/8574 [00:04<00:00, 1918.65it/s]


## Bosch is 1280*720
## Lisa is 1280*960

In [7]:
df_bosch = df_bosch.rename(columns={'Path':'image_id', 'Color':'label'})

In [8]:
## Resizing options
img = cv2.imread(df_bosch['image_id'].iloc[0])
img_new = cv2.resize(img, (1280,960), interpolation=cv2.INTER_CUBIC)

In [9]:
print(img.shape)
print(img_new.shape)

(720, 1280, 3)
(960, 1280, 3)


In [10]:
## Resizing respective boxes
# Get the scaling factor
img_shape = (720, 1280)
reshaped_img_shape = (960, 1280)

#the scaling factor = (960/720, 1280/1280)
scale = np.flipud(np.divide(reshaped_img_shape, img_shape))  # you have to flip because the image.shape is (y,x) but your corner points are (x,y)
x_min = 633
y_min = 342
x_max = 638
y_max = 351
#use this on to get new top left corner and bottom right corner coordinates
new_x_min = np.multiply(x_min, scale )
new_x_max = np.multiply(x_max, scale )
new_y_min = np.multiply(y_min, scale )
new_y_max = np.multiply(y_max, scale )

In [11]:
print(int(new_x_min[1]))
print(int(new_x_max[1]))
print(int(new_y_max[1]))
print(int(new_y_min[1]))

844
850
468
456
